In [1]:
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121, DenseNet201
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
# **本地用matplotlib绘图可以，但是在Linux服务器运行代码绘图的时候会报错，需要添加这行代码**
# plt.switch_backend('agg')

# os.environ["CUDA_VISIBLE_DEVICES"] = '2,3,4'

train_dir = 'data/train/'

IMG_SIZE = (96, 96)
IN_SHAPE = (96, 96, 3)

dropout_rate = 0.5

def load_data(train, valid):
    train_datagen = ImageDataGenerator(preprocessing_function=lambda x: (x - x.mean()) / x.std() if x.std() > 0 else x,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    valid_datagen = ImageDataGenerator(preprocessing_function=lambda x: (
        x - x.mean()) / x.std() if x.std() > 0 else x)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        directory=train_dir,
        x_col='id',
        y_col='label',
        has_ext=False,
        # subset='training',
        batch_size=8,
        seed=2019,
        shuffle=True,
        class_mode='binary',
        target_size=IMG_SIZE)

    valid_generator = valid_datagen.flow_from_dataframe(
        dataframe=valid,
        directory=train_dir,
        x_col='id',
        y_col='label',
        has_ext=False,
        # subset='validation',
        batch_size=8,
        seed=2019,
        shuffle=False,
        class_mode='binary',
        target_size=IMG_SIZE
    )
    print(len(train_generator))
    return train_generator, valid_generator

In [3]:
def model_bulid():
    inputs = Input(IN_SHAPE)
    conv_base = DenseNet201(
        weights='imagenet',
        include_top=False,
        input_shape=IN_SHAPE
    )
    x = conv_base(inputs)
    out = Flatten()(x)
    # **如果换成256验证集效果很差，我也不知道为什么**
    out = Dense(512)(out)
    out = BatchNormalization()(out)
    out = Activation(activation='relu')(out)
    out = Dropout(dropout_rate)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model = Model(inputs, out)

    conv_base.Trainable = True
    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == 'res5a_branch2a':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    model.compile(Adam(),
                  loss="binary_crossentropy", metrics=["accuracy"])

    model.summary()
#     plot_model(model, to_file='NetStruct.png', show_shapes=True)
    return model

In [ ]:
def train_model():

    nfolds = 5
    epoch = 30
    random_state = 2019

    df_train = pd.read_csv('data/train_labels.csv')
    df_train = df_train.sample(n=20, random_state=random_state)
    df_train = df_train.values
    kf = KFold(n_splits=nfolds,
               shuffle=True, random_state=random_state)
    num_fold = 0
    for train_index, valid_index in kf.split(df_train):
        
        train_df, valid_df = df_train[train_index], df_train[valid_index]
        train_df = pd.DataFrame(train_df, columns=['id', 'label']).astype('str')
        valid_df = pd.DataFrame(valid_df, columns=['id', 'label']).astype('str')
        print(train_df.head())
        print(valid_df.head())
        train, valid = load_data(train_df, valid_df)
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(train))
        print('Split valid: ', len(valid))
        model = model_bulid()
        tensorboard = TensorBoard(log_dir='./logs_kfold',  # log 目录
                                  # histogram_freq=1,  # 按照何等频率（epoch）来计算直方图，0为不计算
                                  # batch_size=batch_size,     # 用多大量的数据计算直方图
                                  write_graph=True,  # 是否存储网络结构图
                                  write_grads=False,  # 是否可视化梯度直方图
                                  write_images=False,  # 是否可视化参数
                                  embeddings_freq=0,
                                  embeddings_layer_names=None,
                                  embeddings_metadata=None)
        model_checkpoint = ModelCheckpoint(
            'weights_kfold'+'_'+str(num_fold)+'.h5', monitor='val_loss', save_best_only=True)
        earlystopper = EarlyStopping(
            monitor='val_loss', patience=2, verbose=1)
        reducel = ReduceLROnPlateau(
            monitor='val_loss', patience=1, verbose=1, factor=0.1)

        history = model.fit_generator(train,
                                      validation_data=valid,
                                      epochs=epoch,
                                      callbacks=[reducel, earlystopper, model_checkpoint, tensorboard],
                                      steps_per_epoch=train_df.shape[0]/8,
                                      validation_steps=valid_df.shape[0]/8)

        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='valid')
        plt.title("model loss")
        plt.ylabel("loss")
        plt.xlabel("epoch")
        plt.legend(["train", "valid"], loc="upper left")
        plt.savefig('loss_performance'+'_'+str(num_fold)+'.png')
        plt.clf()
        plt.plot(history.history['acc'], label='train')
        plt.plot(history.history['val_acc'], label='valid')
        plt.title("model acc")
        plt.ylabel("acc")
        plt.xlabel("epoch")
        plt.legend(["train", "valid"], loc="upper left")
        plt.savefig('acc_performance'+'_'+str(num_fold)+'.png')

        with open('logs_kfold.txt', 'a+') as f:
            f.write(str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"))+'\n')
            f.write(str(num_fold)+'\n')
            f.write(str(history.history['loss'])+'\n')
            f.write(str(history.history['val_loss'])+'\n')
            f.write(str(history.history['acc'])+'\n')
            f.write(str(history.history['val_acc'])+'\n')


train_model()



                                         id label
0  1baf53d98f7d4721e3a491011cb2bf2463c5d29b     0
1  e9644bfb5f3072d3038e9ad74f0cf741af690581     1
2  31234755b94e3e4ac916b8ce2242b5719c8ba403     1
3  86abd7205c7e5cef8fef44d530c9785850e3062b     1
4  97c5c61a5ab8223b0fb42b69f2ac24e02d5b4416     0
                                         id label
0  7653fda2957b0831b28c78d18e64637ae2bd44d9     0
1  7232ca54dbc27c520410046caf97544d2ecff0bd     1
2  edf83f14fdec3b12d1c084b0d21fc2d384589631     0
3  3fdb04c3b6f555e14660b5f21fe0fe69b1ff5f35     1
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Start KFold number 1 from 5
Split train:  0
Split valid:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
densenet201 (Model)          (None, 3, 3, 1920) 